In [1]:
using StanModels
using CmdStan, StanMCMCChain

ProjDir = rel_path_s("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_2 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.2",  model=m12_6_2,
output_format=:mcmcchain);


File /Users/rob/.julia/dev/StanModels/notebooks/12/tmp/m12.6.2.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_2_data = Dict("N" => size(d, 1), "T" => d[:total_tools],
"N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_2_data, ProjDir,
diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/notebooks/12/tmp/m12.6.2.stan' at line 19)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD         Naive SE        MCSE         ESS    
         lp__ -39.291259450  3.0090714725 0.047577597477 0.1028879647  855.334234
accept_stat__   0.940581443  0.0821686425 0.001299200312 0.0013856502 1000.000000
   stepsize__   0.052082025  0.0031226338 0.000049373175 0.0004999591   39.009752
  treedepth__   5.802000000  0.4629790356 0.007320341307 0.0146550070  998.046073
 n_leapfrog__  67.500000000 27.2064523186 0.430171781897 0.6502559654 1000.000000
  divergent__   0.000000000  0.0000000000 0.000000000000 0.0000000000         NaN
     energy__  45.803302450  3.8858758113 0.061441091342 0.1427906616  740.590645
        alpha   1.103577719  0.7182322968 0.011356249735 0.0249645257  827.719564
  a_society.1  -0.199922703  0.2313128221 0.003657376850 0.0049205005 1000.000000
  a_society.2   0.044449418  0.2110678317 0.003337275445 0.00509

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*